In [69]:
import sys
sys.path.append("../")


import os
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrices
from openpyxl import load_workbook
import engarde.decorators as ed

from library import test
@ed.verify(test.allyearsandtpsd)
@ed.verify(test.alldois)
def load(data):
    return data

In [79]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/impact/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'cits_match.csv'),
                  sep=",", low_memory= False)
load(data)
data.head()

,Unnamed: 0,Unnamed: 0.1,district,distname,year,cntyname,distischarter,rating_academic,rating_financial,eligible,...,scores12,scores13,scores14,scores15,ps1,psweight1,pswtshare1,ps2,psweight2,pswtshare2
0,0,0,1902,CAYUGA ISD,2012,ANDERSON,N,NaN,NaN,NaN,...,0.376702,0.687344,0.698003,0.766679,0.891787,1.121344,0.109808,0.911461,1.097139,0.107865
1,1,1,1903,ELKHART ISD,2012,ANDERSON,N,NaN,NaN,NaN,...,-0.059983,0.127778,0.322875,0.552578,0.918142,1.089157,0.106656,0.919345,1.087731,0.106940
2,2,2,1904,FRANKSTON ISD,2012,ANDERSON,N,NaN,NaN,NaN,...,-0.193364,0.201075,0.257979,0.530431,0.870107,1.149283,0.112544,0.864370,1.156913,0.113742
3,3,3,1906,NECHES ISD,2012,ANDERSON,N,NaN,NaN,NaN,...,0.785917,0.873445,0.883988,0.274774,0.878874,1.137820,0.111421,0.921676,1.084980,0.106670
4,4,4,1907,PALESTINE ISD,2012,ANDERSON,N,NaN,NaN,NaN,...,-0.580664,-0.538253,-0.363930,-0.255851,0.843056,1.186161,0.116155,0.815476,1.226277,0.120561


In [71]:
def coef_with_stars(coef, pvalue):
    if pvalue >.05:
        coef = str(coef)
    if pvalue <= .05:
        coef = str(coef) + '*'
    if pvalue <= .01:
        coef = coef + '*'
    if pvalue <= .001:
        coef = coef + '*'
    return(coef)
test = coef_with_stars(.1, .005)

In [72]:
data.district.nunique()

1029

In [73]:
data[pd.notnull(data.year_centered)].district.nunique()

981

# Drop districts with missing doi_year and missing pscores

In [74]:
reg_vars = ['year_centered', 'avescores', 'treat', 'psweight1', 'psweight2']
data = data.dropna(subset=reg_vars)

# No Matching

In [58]:
y, X = dmatrices('avescores ~  treat + year_centered + treatyear + yearpost1 + treatpostyear1 + yearpost2 + treatpostyear2 + yearpost3 + treatpostyear3',
                 data=data, return_type='dataframe', NA_action = "drop")
cluster = data.dropna(subset=reg_vars).district
mod = smf.OLS(y, X)    # Describe model
res = mod.fit(cov_type = 'cluster',  cov_kwds={'groups': pd.DataFrame(cluster)})       # Fit model
print(res.summary())   # Summarize model
row_1 = []
for coef in ['treat', 'year_centered', 'treatyear', 'yearpost1', 'treatpostyear1']:
    row_1.append(coef_with_stars(res.params[coef].round(2), res.pvalues[coef]))
    row_1.append('(' + str(res.bse[coef].round(2)) + ')')
row_1.append('')
row_1.append('')
row_1.append(res.rsquared_adj.round(2))
row_1

                            OLS Regression Results                            
Dep. Variable:              avescores   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     138.3
Date:                Sat, 27 Apr 2019   Prob (F-statistic):          4.62e-141
Time:                        14:27:15   Log-Likelihood:                -7297.6
No. Observations:                6793   AIC:                         1.461e+04
Df Residuals:                    6785   BIC:                         1.467e+04
Df Model:                           7                                         
Covariance Type:              cluster                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.0773      0.051     -1.

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1633: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/base/model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 7
  'rank is %d' % (J, J_), ValueWarning)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWar

['0.5***',
 '(0.06)',
 '0.1***',
 '(0.01)',
 '-0.0',
 '(0.01)',
 '-0.08**',
 '(0.03)',
 '0.02',
 '(0.01)',
 '',
 '',
 0.12]

# Matching without pre-tests


In [59]:
y, X = dmatrices('avescores ~  treat + year_centered + treatyear + yearpost1 + treatpostyear1 + yearpost2 + treatpostyear2 + yearpost3 + treatpostyear3',
                 data=data, return_type='dataframe', NA_action = "drop")
cluster = data.dropna(subset=reg_vars).district
psweights = data.dropna(subset=reg_vars).psweight1
mod = smf.WLS(y, X, weights = psweights)    # Describe model
res = mod.fit(cov_type = 'cluster',  cov_kwds={'groups': pd.DataFrame(cluster)})       # Fit model
print(res.summary())   # Summarize model
row_2 = []
for coef in ['treat', 'year_centered', 'treatyear', 'yearpost1', 'treatpostyear1']:
    row_2.append(coef_with_stars(res.params[coef].round(2), res.pvalues[coef]))
    row_2.append('(' + str(res.bse[coef].round(2)) + ')')
row_2.append('X')
row_2.append('')
row_2.append(res.rsquared_adj.round(2))
row_2

                            WLS Regression Results                            
Dep. Variable:              avescores   R-squared:                       0.094
Model:                            WLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     118.0
Date:                Sat, 27 Apr 2019   Prob (F-statistic):          4.31e-125
Time:                        14:27:21   Log-Likelihood:                -8140.4
No. Observations:                6793   AIC:                         1.630e+04
Df Residuals:                    6785   BIC:                         1.635e+04
Df Model:                           7                                         
Covariance Type:              cluster                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.0604      0.058      1.

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1633: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/base/model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 7
  'rank is %d' % (J, J_), ValueWarning)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWar

['0.33***',
 '(0.06)',
 '0.1***',
 '(0.01)',
 '-0.0',
 '(0.01)',
 '-0.1**',
 '(0.03)',
 '0.02',
 '(0.01)',
 'X',
 '',
 0.09]

# Matching with Pre-tests

In [60]:
data = data.dropna(subset = ['avescores', 'psweight2', 'scores15', 'scores14', 'scores13', 'scores12'], how = 'any')
y, X = dmatrices('avescores ~  treat + year_centered + treatyear + yearpost1 + treatpostyear1 + yearpost2 + treatpostyear2 + yearpost3 + treatpostyear3',
                 data=data, return_type='dataframe', NA_action = "drop")
cluster = data[pd.notnull(data.scores14)].distname
psweights = data[pd.notnull(data.scores14)].psweight2
mod = smf.WLS(y, X, weights = psweights)    # Describe model
res = mod.fit(cov_type = 'cluster',  cov_kwds={'groups': pd.DataFrame(cluster)})       # Fit model
print(res.summary())   # Summarize model
row_3 = []
for coef in ['treat', 'year_centered', 'treatyear', 'yearpost1', 'treatpostyear1']:
    row_3.append(coef_with_stars(res.params[coef].round(2), res.pvalues[coef]))
    row_3.append('(' + str(res.bse[coef].round(2)) + ')')
row_3.append('X')
row_3.append('X')
row_3.append(res.rsquared_adj.round(2))
row_3

                            WLS Regression Results                            
Dep. Variable:              avescores   R-squared:                       0.071
Model:                            WLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     107.0
Date:                Sat, 27 Apr 2019   Prob (F-statistic):          4.94e-116
Time:                        14:27:36   Log-Likelihood:                -8180.8
No. Observations:                6793   AIC:                         1.638e+04
Df Residuals:                    6785   BIC:                         1.643e+04
Df Model:                           7                                         
Covariance Type:              cluster                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.2293      0.059      3.

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1633: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/base/model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 7
  'rank is %d' % (J, J_), ValueWarning)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWar

['0.12',
 '(0.07)',
 '0.1***',
 '(0.01)',
 '-0.0',
 '(0.01)',
 '-0.09***',
 '(0.03)',
 '0.02',
 '(0.01)',
 'X',
 'X',
 0.07]

In [61]:
df = pd.DataFrame(
    {'Model 1': row_1,
    'Model 2': row_2,
    'Model 3': row_3})
df

,Model 1,Model 2,Model 3
0,0.5***,0.33***,0.12
1,(0.06),(0.06),(0.07)
2,0.1***,0.1***,0.1***
3,(0.01),(0.01),(0.01)
4,-0.0,-0.0,-0.0
5,(0.01),(0.01),(0.01)
6,-0.08**,-0.1**,-0.09***
7,(0.03),(0.03),(0.03)
8,0.02,0.02,0.02
9,(0.01),(0.01),(0.01)


In [62]:
file = table_path + 'results_main.xlsx'
wb = load_workbook(file)
ws = wb.active
df
for model, col in zip(['Model 1', 'Model 2', 'Model 3'], [2,3,4]):
    row_n = 3
    for ob in df[model]:
        ws.cell(row=row_n, column=col).value = ob
        row_n = row_n + 1
wb.save(file)

# Fixed Effects

In [91]:
data_fe = pd.get_dummies(data = data, columns = ['year'])
reg_vars = ['year_centered', 'avescores', 'treat', 'psweight1', 'psweight2']
data_fe = data_fe.dropna(subset=reg_vars)
#data_fe = data_fe.dropna(reg_vars)
data_fe

,Unnamed: 0,Unnamed: 0.1,district,distname,cntyname,distischarter,rating_academic,rating_financial,eligible,type,...,ps2,psweight2,pswtshare2,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,0,0,1902,CAYUGA ISD,ANDERSON,N,NaN,NaN,NaN,H,...,0.911461,1.097139,0.107865,1,0,0,0,0,0,0
1,1,1,1903,ELKHART ISD,ANDERSON,N,NaN,NaN,NaN,G,...,0.919345,1.087731,0.106940,1,0,0,0,0,0,0
2,2,2,1904,FRANKSTON ISD,ANDERSON,N,NaN,NaN,NaN,H,...,0.864370,1.156913,0.113742,1,0,0,0,0,0,0
3,3,3,1906,NECHES ISD,ANDERSON,N,NaN,NaN,NaN,H,...,0.921676,1.084980,0.106670,1,0,0,0,0,0,0
4,4,4,1907,PALESTINE ISD,ANDERSON,N,NaN,NaN,NaN,E,...,0.815476,1.226277,0.120561,1,0,0,0,0,0,0
5,5,5,1908,WESTWOOD ISD,ANDERSON,N,NaN,NaN,NaN,G,...,0.867627,1.152569,0.113315,1,0,0,0,0,0,0
6,6,6,1909,SLOCUM ISD,ANDERSON,N,NaN,NaN,NaN,H,...,0.891094,1.122216,0.110331,1,0,0,0,0,0,0
7,7,7,2901,ANDREWS ISD,ANDREWS,N,NaN,NaN,NaN,D,...,0.730185,1.369516,0.134644,1,0,0,0,0,0,0
8,8,9,3902,HUDSON ISD,ANGELINA,N,NaN,NaN,NaN,G,...,0.931049,1.074058,0.105596,1,0,0,0,0,0,0
9,9,10,3903,LUFKIN ISD,ANGELINA,N,NaN,NaN,NaN,E,...,0.865588,1.155284,0.113582,1,0,0,0,0,0,0


In [110]:
y, X = dmatrices('avescores ~  + doi + treatpost +  year_2013 + year_2014 + year_2015 + year_2016 + year_2017 + year_2018',
                 data=data_fe, return_type='dataframe', NA_action = "drop")
cluster = data_fe.dropna(subset=reg_vars).district
mod = smf.OLS(y, X)    # Describe model
res = mod.fit(cov_type = 'cluster',  cov_kwds={'groups': pd.DataFrame(cluster)})       # Fit model
print(res.summary())   # Summarize model


                            OLS Regression Results                            
Dep. Variable:              avescores   R-squared:                       0.115
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     289.8
Date:                Sat, 27 Apr 2019   Prob (F-statistic):          4.87e-251
Time:                        15:36:10   Log-Likelihood:                -7309.4
No. Observations:                6793   AIC:                         1.464e+04
Df Residuals:                    6784   BIC:                         1.470e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.4569      0.047     -9.741      